# Fix Poor Predictions from Comprehend Custom Text Classifier

In [1]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

In [2]:
import io
import json
import uuid
import time
import boto3
import botocore

# Amazon Python SDK clients
sagemaker = boto3.client('sagemaker', region)
comprehend = boto3.client('comprehend', region)
a2i = boto3.client('sagemaker-a2i-runtime')
s3 = boto3.client('s3', region)

# Retrieve the `augmented_ai_flow_definition_arn` Created Previously

In [3]:
%store -r augmented_ai_flow_definition_arn

no stored variable or alias augmented_ai_flow_definition_arn


In [4]:
print(augmented_ai_flow_definition_arn)

NameError: name 'augmented_ai_flow_definition_arn' is not defined

# _Wait for the Comprehend Job to Complete from the Previous Section_

# Retrieve the `comprehend_endpoint_arn` Deployed Previously

In [5]:
%store -r comprehend_endpoint_arn

no stored variable or alias comprehend_endpoint_arn


In [6]:
try: 
    comprehend_endpoint_arn
except NameError:
    print('*** PLEASE WAIT FOR THE Comprehend JOB TO FINISH IN THE PREVIOUS SECTION BEFORE CONTINUING ***')
    print('*** YOU WILL NEED TO RESTART THIS NOTEBOOK ONCE THE JOB FINISHES ***')

*** PLEASE WAIT FOR THE Comprehend JOB TO FINISH IN THE PREVIOUS SECTION BEFORE CONTINUING ***
*** YOU WILL NEED TO RESTART THIS NOTEBOOK ONCE THE JOB FINISHES ***


In [7]:
print(comprehend_endpoint_arn)

NameError: name 'comprehend_endpoint_arn' is not defined

# Check the Confidence Score for Each Comprehend Prediction
If < threshold, start the human loop.  You can integrate this type of logic into your application using the SDK.  In this case, we're using the Python SDK.

# Use Comprehend to Predict Some Sample Reviews

In [8]:
sample_reviews = [
                  'I enjoy this product', 
                  'I am unhappy with this product', 
                  'It is okay', 
                  'sometimes it works'
                 ]

# Start a Human Loop When Comprehend Does Not Produce a Confident Prediction
The human lmoop will engage our workforce for human review if the confidence of the Comprehend prediction is less than the provided confidence.

![](img/augmented-ai-comprehend-predictions.png)

In [9]:
human_loops_started = []

CONFIDENCE_SCORE_THRESHOLD = 0.90

for sample_review in sample_reviews:
    # Call the Comprehend Custom model that we trained earlier
    response = comprehend.classify_document(Text=sample_review, 
                                            EndpointArn=comprehend_endpoint_arn)

    star_rating = response['Classes'][0]['Name']
    confidence_score = response['Classes'][0]['Score']
    
    print(f'Processing sample_review: \"{sample_review}\"')
    
    # Our condition for when we want to engage a human for review
    if (confidence_score < CONFIDENCE_SCORE_THRESHOLD):
    
        humanLoopName = str(uuid.uuid4())
        inputContent = {
            'initialValue': star_rating,
            'taskObject': sample_review
        }
        start_loop_response = a2i.start_human_loop(
            HumanLoopName=humanLoopName,
            FlowDefinitionArn=augmented_ai_flow_definition_arn,
            HumanLoopInput={
                'InputContent': json.dumps(inputContent)
            }
        )

        human_loops_started.append(humanLoopName)

        print(f'Confidence score of {confidence_score} for star rating of {star_rating} is less than the threshold of {CONFIDENCE_SCORE_THRESHOLD}')
        print(f'*** ==> Starting human loop with name: {humanLoopName}  \n')
    else:
        print(f'Confidence score of {confidence_score} for star rating of {star_rating} is above threshold of {CONFIDENCE_SCORE_THRESHOLD}')
        print('No human loop created. \n')

NameError: name 'comprehend_endpoint_arn' is not defined

# Check Status of Human Loop

In [10]:
completed_human_loops = []
for human_loop_name in human_loops_started:
    resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    print(f'HumanLoop Name: {human_loop_name}')
    print(f'HumanLoop Status: {resp["HumanLoopStatus"]}')
    print(f'HumanLoop Output Destination: {resp["HumanLoopOutput"]}')
    print('')
        
    if resp['HumanLoopStatus'] == 'Completed':
        completed_human_loops.append(resp)

# Wait For Workers to Complete Their Human Loop Tasks

Navigate to the link below and login with your email and password that you used when you set up the Private Workforce.

In [11]:
%store -r augmented_ai_workteam_arn

no stored variable or alias augmented_ai_workteam_arn


In [12]:
print(augmented_ai_workteam_arn)

NameError: name 'augmented_ai_workteam_arn' is not defined

In [13]:
workteam_name = augmented_ai_workteam_arn[augmented_ai_workteam_arn.rfind('/') + 1:]
print(workteam_name)
print('Navigate to the private worker portal and complete the human loop.')
print('Make sure you have invited yourself to the workteam and received the signup email.')
print('Note:  Check your spam filter if you have not received the email.')
print('')
print('https://' + sagemaker.describe_workteam(WorkteamName=workteam_name)['Workteam']['SubDomain'])

NameError: name 'augmented_ai_workteam_arn' is not defined

# Start Labeling

<img src="img/augmented-comprehend-custom-start-working.png" width="80%" align="left">

# Select Label

<img src="img/augmented-comprehend-custom-select-label.png" width="80%" align="left">

# Loop is Completed

<img src="img/augmented-comprehend-custom-finished-task.png" width="80%" align="left">

# Verify the Human Loops are Completed

In [14]:
import time

completed_human_loops = []
for human_loop_name in human_loops_started:
    resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    print(f'HumanLoop Name: {human_loop_name}')
    print(f'HumanLoop Status: {resp["HumanLoopStatus"]}')
    print(f'HumanLoop Output Destination: {resp["HumanLoopOutput"]}')
    print('')
    while resp["HumanLoopStatus"] != "Completed":
        print(f'Waiting for HumanLoop to complete.')  
        time.sleep(10)
        resp = a2i.describe_human_loop(HumanLoopName=human_loop_name)
    if resp["HumanLoopStatus"] == "Completed":
        completed_human_loops.append(resp)
        print(f'Completed!')
        print('')

# View Human Labels  

Once the work is complete, Amazon A2I stores the results in the specified S3 bucket and sends a Cloudwatch Event.  Let's check the S3 contents.

In [15]:
import re
import pprint

pp = pprint.PrettyPrinter(indent=4)

fixed_items = []

for resp in completed_human_loops:
    split_string = re.split('s3://' +  bucket + '/', resp['HumanLoopOutput']['OutputS3Uri'])
    output_bucket_key = split_string[1]

    response = s3.get_object(Bucket=bucket, Key=output_bucket_key)
    content = response['Body'].read().decode('utf-8')
    json_output = json.loads(content)
    print(json_output)

    input_content = json_output['inputContent']
    human_answer = json_output['humanAnswers'][0]['answerContent']
    fixed_item = {'input_content': input_content, 'human_answer': human_answer}
    fixed_items.append(fixed_item)

# Prepare the Data for Re-training

In [16]:
df_fixed_items = pd.DataFrame(fixed_items)

In [17]:
df_fixed_items.head()

""


# Once finished, delete the Comprehend Custom Model Endpoint

In [18]:
# comprehend.delete_endpoint(EndpointArn=comprehend_endpoint_arn)

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();